In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/training-data-for-multiability-chabot/final_training_data (1).csv
/kaggle/input/training-data-for-multiability-chabot/final_training_data.csv


In [2]:
!pip install -q -U transformers
!pip install -q -U accelarate
!pip install -q -U bitsandbytes
!pip install -q -U trl
!pip install -q -U peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 76.5 MB/s eta 0:00:00:00:010:01
ERROR: Could not find a version that satisfies the requirement accelarate (from versions: none)
ERROR: No matching distribution found for accelarate
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 18.0 MB/s eta 0:00:0000:01:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.8/410.8 kB 14.2 MB/s eta 0:00:00


In [3]:
model_name = "microsoft/Phi-3.5-mini-instruct"
dataset_path = "/kaggle/input/training-data-for-multiability-chabot/final_training_data.csv"

In [4]:
import os
import warnings
import pandas as pd 
import numpy as np 
import torch 
import torch.nn as nn

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments, pipeline, logging
from peft import LoraConfig, PeftConfig
from trl import SFTTrainer, setup_chat_format

In [5]:
data = pd.read_csv('/kaggle/input/training-data-for-multiability-chabot/final_training_data.csv')

In [6]:
data.head(10)

,Unnamed: 0,prompt,response,category,category_id
0,240,"Hi doctor,I am just wondering what is abutting...",Hi. I have gone through your query with dilige...,HC,0
1,620,"Hello doctor, I have a stiffness of neck and p...","Hello. From your symptoms, it seems like fibro...",HC,0
2,1208,"Hi doctor, I am a 26 year old male. I am 5 fee...",Hello. I have gone through your information an...,HC,0
3,1388,"Hi doctor, I am a 26 year old male. I am 5 fee...",Hello. I have gone through your information an...,HC,0
4,1828,"Hello doctor, I am a 24-year-old male. I am af...",Hello. You can try to retract foreskin daily d...,HC,0
5,1855,"Hello doctor, I had a sudden weight gain. Are ...","Hello. Avoid raw cabbage, cauliflower, brocoll...",HC,0
6,2738,"Hi doctor, I am 22 years old. I have been faci...",Hi. With the symptoms I feel sorry for your he...,HC,0
7,3048,"Hello doctor, My friend aged 30 had two drops ...",Hi. I want to assure you not to worry as every...,HC,0
8,3272,"Hello doctor, I had symptoms of cold, cough an...",Hello. Staphylococcus aureus is a normal flora...,HC,0
9,3404,"Hi doctor, I am a lactating mother. I feel diz...",Hi. Your blood pressure (BP) is absolutely nor...,HC,0


In [7]:
data['category'].value_counts()

category
HC    400
TC    400
EC    260
BA    145
Name: count, dtype: int64

In [8]:
label_mapping = {
    'HC': 'Medical',
    'TC': 'Telecom',
    'BA': 'Banking',
    'EC': 'Ecommerce'
}

# Replace the labels
data['category'] = data['category'].replace(label_mapping)

print(data)

      Unnamed: 0                                             prompt  \
0            240  Hi doctor,I am just wondering what is abutting...   
1            620  Hello doctor, I have a stiffness of neck and p...   
2           1208  Hi doctor, I am a 26 year old male. I am 5 fee...   
3           1388  Hi doctor, I am a 26 year old male. I am 5 fee...   
4           1828  Hello doctor, I am a 24-year-old male. I am af...   
...          ...                                                ...   
1200      283316  Can I get help with understanding my account s...   
1201      283317  What should I do if I have a question about my...   
1202      283318  What are your policies on account inactivity f...   
1203      283319                                                 Hi   
1204      283320                        Tell me about your services   

                                               response category  category_id  
0     Hi. I have gone through your query with dilige...  Medical   

In [9]:
from datasets import Dataset
data = Dataset.from_pandas(data)


print(data)

Dataset({
    features: ['Unnamed: 0', 'prompt', 'response', 'category', 'category_id'],
    num_rows: 1205
})


In [10]:
columns_to_remove = ["Unnamed: 0", "category_id"]  # Replace with the names of the columns you want to drop
data = data.map(lambda x: x, remove_columns=columns_to_remove)
print(data)

Map:   0%|          | 0/1205 [00:00<?, ? examples/s]

Dataset({
    features: ['prompt', 'response', 'category'],
    num_rows: 1205
})


In [11]:
compute_dtype = getattr(torch, 'float16')

bnbconfig = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=True,
)


model = AutoModelForCausalLM.from_pretrained(
    model_name, 
    torch_dtype=compute_dtype, 
    low_cpu_mem_usage=True,
    #trust_remote_code=True,
)

config.json:   0%|          | 0.00/3.45k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/16.3k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/195 [00:00<?, ?B/s]

In [12]:
tokenizer = AutoTokenizer.from_pretrained(model_name)


model, tokenizer = setup_chat_format(model, tokenizer)

tokenizer_config.json:   0%|          | 0.00/3.98k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

ValueError: Chat template is already added to the tokenizer. If you want to overwrite it, please set it to None

In [13]:
def format_chat_template(row):
    row_json = [{"role": "user", "content": row["prompt"]},
               {"role": "assistant", "content": row['response']}]
    row["text"] = tokenizer.apply_chat_template(row_json, tokenize=False)
    return row

dataset = data.map(
    format_chat_template,
    num_proc=4,
)

dataset['text'][5]

Map (num_proc=4):   0%|          | 0/1205 [00:00<?, ? examples/s]

'<|user|>\nHello doctor, I had a sudden weight gain. Are there any foods that should be avoided either at certain times of the day or all together while taking Levothyroxine to treat an underactive thyroid?<|end|>\n<|assistant|>\nHello. Avoid raw cabbage, cauliflower, brocolli, soy and soy products like soy sauce, soy milk, tofu, strawberry, walnut, spinach, and brussel sprouts. Do not take any meals or drink except plain water for atleast 45 minutes after taking medication. Avoid taking acidity-causing foods, calcium and iron supplements with the medicine.<|end|>\n<|endoftext|>'

In [14]:
training_arguments = TrainingArguments(
    output_dir="RANDOM_CHATBOT",
    num_train_epochs=1,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    optim="paged_adamw_32bit",
    save_steps=0,
    logging_steps=15,
    learning_rate=5e-4,
    weight_decay=0.001,
    fp16=True,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    lr_scheduler_type="cosine",
    report_to="none",
)

In [15]:
from peft import get_peft_model, prepare_model_for_kbit_training, PeftModel
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['up_proj', 'down_proj', 'gate_proj', 'k_proj', 'q_proj', 'v_proj', 'o_proj']
)
model = get_peft_model(model, peft_config)

In [16]:
trainer = SFTTrainer(
    model = model, 
    train_dataset = dataset, 
    peft_config = peft_config, 
    tokenizer = tokenizer, 
    args = training_arguments,
    #dataset_text_field="generated_prompt",
)

trainer.train()

<ipython-input-16-5bb1f8203460>:1: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(


Converting train dataset to ChatML:   0%|          | 0/1205 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/1205 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/1205 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/1205 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


RuntimeError: chunk expects at least a 1-dimensional tensor

In [17]:
messages = [
    {
        "role": "user",
        "content": "I am have been charged for 40gb of data while i used 30gb data only. Please resolve this issue"
    }
]

prompt = tokenizer.apply_chat_template(messages, tokenize=False, 
                                       add_generation_prompt=True)

inputs = tokenizer(prompt, return_tensors='pt', padding=True, 
                   truncation=True).to("cuda")

outputs = model.generate(**inputs, max_length=150, 
                         num_return_sequences=1)

text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(text.split("assistant")[-1])

I am have been charged for 40gb of data while i used 30gb data only. Please resolve this issue I'm an AI and cannot directly resolve issues for you, but I can guide you through the steps to address the problem of being charged for more data than you used:

1. **Review your usage details**: Check your data usage records on your mobile carrier's website or app. Ensure that the reported usage aligns with your actual usage.

2. **Gather evidence**: Collect screenshots or printouts of your usage details, billing statements, and any other relevant information.

3. **Contact your carrier**: Re


In [18]:
model_dir = "Phi3.5-Combined-ChatBot"
trainer.model.save_pretrained(model_dir)
trainer.tokenizer.save_pretrained(model_dir)

Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


('Phi3.5-Combined-ChatBot/tokenizer_config.json',
 'Phi3.5-Combined-ChatBot/special_tokens_map.json',
 'Phi3.5-Combined-ChatBot/tokenizer.model',
 'Phi3.5-Combined-ChatBot/added_tokens.json',
 'Phi3.5-Combined-ChatBot/tokenizer.json')